In [ ]:
import glob
import json
import os
from multiprocessing import Pool, pool

import cv2
import numpy as np
import csv

import pandas as pd
from scipy.io import loadmat
import time
from matplotlib import pyplot as plt

from PIL import Image

<h1> Load sample image and stereo parameters </h1>

In [ ]:

left_image_f = '/root/data/s3/aquabyte-frames-resized-inbound/environment=production/site-id=56/pen-id=88/date=2020-03-01/hour=00/at=2020-03-01T00:00:21.069179000Z/left_frame_crop_660_1878_4004_3000.jpg'
right_image_f = '/root/data/s3/aquabyte-frames-resized-inbound/environment=production/site-id=56/pen-id=88/date=2020-03-01/hour=00/at=2020-03-01T00:00:21.069179000Z/right_frame_crop_276_1860_3652_3000.jpg'


<h1> Rectify the padded image </h1>

In [ ]:
def load_params(params_file):
    params = json.load(open(params_file))
    cameraMatrix1 = np.array(params['CameraParameters1']['IntrinsicMatrix']).transpose()
    cameraMatrix2 = np.array(params['CameraParameters2']['IntrinsicMatrix']).transpose()

    distCoeffs1 = params['CameraParameters1']['RadialDistortion'][0:2] + \
                   params['CameraParameters1']['TangentialDistortion'] + \
                   [params['CameraParameters1']['RadialDistortion'][2]]
    distCoeffs1 = np.array(distCoeffs1)

    distCoeffs2 = params['CameraParameters2']['RadialDistortion'][0:2] + \
                   params['CameraParameters2']['TangentialDistortion'] + \
                   [params['CameraParameters2']['RadialDistortion'][2]]
    distCoeffs2 = np.array(distCoeffs2)

    R = np.array(params['RotationOfCamera2']).transpose()
    T = np.array(params['TranslationOfCamera2']).transpose()

    imageSize = (4096, 3000)
    
    # perform rectification
    (R1, R2, P1, P2, Q, leftROI, rightROI) = cv2.stereoRectify(cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, imageSize, R, T, None, None, None, None, None, cv2.CALIB_ZERO_DISPARITY, 0)

    left_maps = cv2.initUndistortRectifyMap(cameraMatrix1, distCoeffs1, R1, P1, imageSize, cv2.CV_16SC2)
    right_maps = cv2.initUndistortRectifyMap(cameraMatrix2, distCoeffs2, R2, P2, imageSize, cv2.CV_16SC2)
    
    return left_maps, right_maps

def rectify_crop(crop_image_f, side, crop_metadata, stereo_params_f):
    left_maps, right_maps = load_params(stereo_params_f)
    maps = left_maps if side == 'left' else right_maps
    crop = cv2.imread(crop_image_f)
    new_image = np.zeros([3000, 4096, 3]).astype('uint8')
    lower_left = (crop_metadata['y_coord'] + crop_metadata['height'], crop_metadata['x_coord'])
    upper_right = (crop_metadata['y_coord'], crop_metadata['x_coord'] + crop_metadata['width'])
    new_image[upper_right[0]:lower_left[0], lower_left[1]:upper_right[1], :] = np.array(crop)
    remap = cv2.remap(new_image, maps[0], maps[1], cv2.INTER_LANCZOS4)
    nonzero_indices = np.where(remap > 0)
    y_min, y_max = nonzero_indices[0].min(), nonzero_indices[0].max() 
    x_min, x_max = nonzero_indices[1].min(), nonzero_indices[1].max()
    lower_left = (y_max, x_min)
    upper_right = (y_min, x_max)
    print(x_max - x_min)
    print(y_max - y_min)
    print(x_min)
    print(y_min)
    rectified_crop = remap[upper_right[0]:lower_left[0], lower_left[1]:upper_right[1], :].copy()
    cv2.imwrite('/Users/aloksaxena/Desktop/{}_full_res.jpg'.format(os.path.basename(crop_image_f).replace('.jpg', '')), new_image)
    cv2.imwrite('/Users/aloksaxena/Desktop/{}_full_res_rectified.jpg'.format(os.path.basename(crop_image_f).replace('.jpg', '')), remap)
    cv2.imwrite('/Users/aloksaxena/Desktop/{}_rectified.jpg'.format(os.path.basename(crop_image_f).replace('.jpg', '')), rectified_crop)



In [ ]:
stereo_params_f = '/root/data/s3/aquabyte-stereo-parameters/L40039154_R40012648/2020-02-19T03:42:04.748042000Z_L40039154_R40012648_stereo-parameters.json'
left_crop_image_f = '/root/data/s3/aquabyte-frames-resized-inbound/environment=production/site-id=56/pen-id=88/date=2020-03-01/hour=00/at=2020-03-01T00:00:21.069179000Z/left_frame_crop_660_1878_4004_3000.jpg'
left_crop_metadata = {"width": 3344, "height": 1122, "x_coord": 1118, "y_coord": 752, "crop_area": 1100440, "qualityScore": {"quality": 0.003564488375559449, "darkness": 0.9802804589271545, "modelInfo": {"model": "Mobilenet", "input_size": [224, 224, 3], "description": "binary classification good / bad for filtering", "output_size": [3], "probability": {"is_dark": 2, "is_good": 0, "is_blurry": 1}}, "blurriness": 0.01695990562438965}, "mean_luminance": 33.91584729744466}
rectify_crop(left_crop_image_f, 'left', left_crop_metadata, stereo_params_f)

In [ ]:
right_crop_image_f = '/root/data/s3/aquabyte-frames-resized-inbound/environment=production/site-id=56/pen-id=88/date=2020-03-01/hour=00/at=2020-03-01T00:00:21.069179000Z/right_frame_crop_276_1860_3652_3000.jpg'
right_crop_metadata = {"width": 3376, "height": 1140, "x_coord": 622, "y_coord": 752, "crop_area": 1041680, "qualityScore": {"quality": 0.003564488375559449, "darkness": 0.9802804589271545, "modelInfo": {"model": "Mobilenet", "input_size": [224, 224, 3], "description": "binary classification good / bad for filtering", "output_size": [3], "probability": {"is_dark": 2, "is_good": 0, "is_blurry": 1}}, "blurriness": 0.01695990562438965}, "mean_luminance": 32.96295983411412}
rectify_crop(right_crop_image_f, 'right', right_crop_metadata, stereo_params_f)

In [ ]:
left_image = cv2.imread(left_image_f)
right_image = cv2.imread(right_image_f)
cv2.imwrite('/Users/aloksaxena/Desktop/left_crop.jpg', left_image[left_crop_metadata['y_coord']:left_crop_metadata['y_coord']+left_crop_metadata['height'], left_crop_metadata['x_coord']:left_crop_metadata['x_coord']+left_crop_metadata['width'], :])
cv2.imwrite('/Users/aloksaxena/Desktop/right_crop.jpg', right_image[right_crop_metadata['y_coord']:right_crop_metadata['y_coord']+right_crop_metadata['height'], right_crop_metadata['x_coord']:right_crop_metadata['x_coord']+right_crop_metadata['width'], :])
left_crop_image_f = '/Users/aloksaxena/Desktop/left_crop.jpg'
right_crop_image_f = '/Users/aloksaxena/Desktop/right_crop.jpg'
rectify_crop(left_crop_image_f, 'left', left_crop_metadata, stereo_params_f)
rectify_crop(right_crop_image_f, 'right', right_crop_metadata, stereo_params_f)

In [ ]:
im_raw = cv2.imread('/Users/aloksaxena/Downloads/left_frame_crop_734_1362_4096_2710_raw.jpg')
im_rectified = cv2.imread('/Users/aloksaxena/Downloads/left_frame_crop_734_1362_4096_2710_rectified.jpg')

In [ ]:
im_raw.shape


In [ ]:
(im_raw != im_rectified ).sum()

In [ ]:
{"width": 1880, "height": 624, "x_coord": 123, "y_coord": 2177, "crop_area": 1041680, "qualityScore": {"quality": 0.003564488375559449, "darkness": 0.9802804589271545, "modelInfo": {"model": "Mobilenet", "input_size": [224, 224, 3], "description": "binary classification good / bad for filtering", "output_size": [3], "probability": {"is_dark": 2, "is_good": 0, "is_blurry": 1}}, "blurriness": 0.01695990562438965}, "mean_luminance": 32.96295983411412}